In [1]:
"""
This is the function you'll use to add an arbitrary cc table to your running cc history
""";

In [2]:
import pandas as pd
import numpy as np

from openpyxl import load_workbook

import datetime

import glob

## general i/o

In [3]:
def get_dtime (string):
    """
    converts strings to datetime objects using datetime.strptime mostly just wrote this so I could .apply() it in pandas
    method chains
    
    takes (string) which is the full date of format mm/dd/yyyy (can't be 2 digit year)
    """
    return datetime.datetime.strptime(string, '%m/%d/%Y')

In [4]:
def open_full_hist (filepath):
    
    #full cc hist to update
    full_cc_hist = pd.read_csv(filepath)
    full_cc_hist['date'] = full_cc_hist['date'].apply(get_dtime)
    
    return full_cc_hist

## Chase

In [5]:
def get_chase_addendum (glob_list_of_cc_addenda):
    
    chase_file = [z for z in glob_list_of_cc_addenda if 'chase' in z][0]
    
    chase = pd.read_csv(chase_file)
    
    chase_clean = (chase.drop(['Type', 'Post Date'], axis=1)
                   #updated "Trans Date" to "Transaction Date" on 2019-03-05
               .rename(columns = {'Transaction Date' : 'date', 'Description' : 'desc', 'Amount' : 'price'})
               .assign(card = 'chase -8723')
              )
    
    chase_clean['price'] = chase_clean['price'].apply(np.negative)
    chase_clean['date'] = chase_clean['date'].apply(get_dtime)
    
    ii = chase_clean['desc'].str.contains('AUTOMATIC PAYM')
    chase_clean.loc[ii, 'desc'] = 'AUTOMATIC PAYMENT - THANK YOU'
    
#     min_max_dates = [min(chase_clean['date']), max(chase_clean['date'])]
    
    
    return chase_clean, chase_file#, min_max_dates

## Citi

In [6]:
def strip_comma (entry):
    #if the entry is NOT a null (nan)
    if not pd.isnull(entry):

        if ',' in entry:
            
            entry = entry.replace(',', '')

    return entry

In [7]:
def make_price (df):
     
    #work with the credit info
    cre = df['Credit']
    
    #where to flip credit to negative debit
    i = cre.dropna().index
    
        #removed the addition of '-' to the Credits because the statements actually add the negative for me already
        #as of 2019-03-05
    df.loc[i, 'Debit'] = df.loc[i, 'Credit'].astype(str).apply(strip_comma)
    
    #rename for consistent column names
    df = df.rename(columns = {'Debit' : 'price'})
    
    #get rid of commas
    df['price'] = df['price'].astype(str).apply(strip_comma).astype(float)

    df = df.drop('Credit', axis='columns')
    
    
    return df

In [8]:
def fix_date_desc (df):
    
    #using get_dtime from chase munge
    df['date'] = df['date'].apply(get_dtime)
    
    df['desc'] = df['desc'].str.split('\n', expand=True)[0]
    
    df.loc[df['desc'].str.contains('AUTOPAY'), 'desc'] = 'AUTOMATIC PAYMENT - THANK YOU'
    
    return df

In [9]:
def get_citi_addendum (hist_files):
    
    file = [z for z in hist_files if 'citi' in z][0]
    
    df = pd.read_csv(file)
    
    df = make_price(df)
    
    df = df.rename(columns = {'Description' : 'desc', 'Date' : 'date'}).drop('Status', axis = 'columns')
    
    df = fix_date_desc(df)
    
    df['card'] = 'citi -6845'
    
    return df, file

## Discover

In [10]:
def standard_cols (df):
    
    return (df.rename(columns = {'Trans. Date' : 'date', 'Description' : 'desc', 'Amount' : 'price', 'Category' : 'cat'})
            .drop('Post Date', axis='columns')
           )

In [11]:
def replace_autopay (df):
    
    i = df['desc'].str.contains('DIRECTPAY')
    
    df.loc[i, 'desc'] = 'AUTOMATIC PAYMENT - THANK YOU'
    
    return df

In [12]:
def get_disc_addendum (hist_files):
    
    file = [z for z in hist_files if 'discover' in z][0]
    
    df = pd.read_csv(file)
    
    df = standard_cols(df)
    
    df = replace_autopay(df)
    
    df['date'] = df['date'].apply(get_dtime)
    
    df['card'] = 'discover -1362'
    
    return df, file

# The work

In [13]:
def find_new_entries (full_cc_hist, clean_new_entries, card_name):
    """
    Takes a properly column titled and formatted df of possible new entries
    and takes all new entries after the last recorded transaction in the history,
    returning these as a df
    """
    
    #get this card's transactions
    card_idx = full_cc_hist['card'] == card_name

    #get the date of the last recorded transaction on this card in the history
    last_recorded_trans_date = max(full_cc_hist.loc[card_idx, 'date'])
    

    #get all downloaded transactions that happened AFTER the last recorded transaction date
    #this assumed no retroactively applied transactions
    eligible_new_trans = clean_new_entries.loc[clean_new_entries['date'] >= last_recorded_trans_date]
    

    #get the transactions from the full hist that occured on the last recorded date
    #some of these may be repeated in the new entries, but some new entries may be new from that date
    last_recorded_trans = full_cc_hist.loc[(full_cc_hist['date'] == last_recorded_trans_date) & card_idx]

    #only possible overlaps here are ON the latest recorded transaction day, so put
    #the new, and existing transactions from that day, together and remove duplicates
    possible_overlap = pd.concat([eligible_new_trans, last_recorded_trans], sort=False)
    

    new_entries_idx = ~possible_overlap.duplicated(['date', 'price', 'card'], keep=False)
    
    #adding this block because a unique (not duplicated) last_recorded_trans will show up as a new transaction
    #thanks to the concat step above followed by NOT dropping it again with the indexing below
    #if it is not duplicated in the new entries, it is just ignored in the if part of this if/else
    if all(new_entries_idx): #this means NONE of the eligible new transactions are duplicates of something in the last recorded transactions
        new_entries = eligible_new_trans
    else: #this means there were duplicates, both/all of which will be dropped
        new_entries = possible_overlap.loc[new_entries_idx]
    
    return new_entries

In [14]:
def all_things_to_add (hist_files, full_cc_hist):
    
    #get the properly formatted chase transactions to be added and the date range they span
    chase_clean, chase_file = get_chase_addendum(hist_files)
    new_chase = find_new_entries (full_cc_hist, chase_clean, 'chase -8723')
#     print('CHASE', new_chase)
    
    citi_clean, citi_file = get_citi_addendum(hist_files)
    new_citi = find_new_entries (full_cc_hist, citi_clean, 'citi -6845')
#     print('CITI', new_citi)
    
    disc_clean, disc_file = get_disc_addendum(hist_files)
    new_disc = find_new_entries (full_cc_hist, disc_clean, 'discover -1362')
#     print('DISC', new_disc)
    
    return pd.concat([new_chase, new_citi, new_disc], sort=False), [chase_file, citi_file, disc_file], [chase_clean, citi_clean, disc_clean]

In [15]:
def save_new_cc_hist (full_cc_hist, all_new_entries, file_dir, files_processed):
    
    new = (pd.concat([full_cc_hist, all_new_entries], sort=False)
           .sort_values(by='date')
           .reset_index(drop=True)
          )
    
    #this requires that the cc history is in the same place as the files with new entries and has the name
    # 'cc_hist_tidy'
    new.to_csv(file_dir + '/cc_hist_tidy' + '.csv', index=False)
    
    print("The updated history of cc transactions has been saved to file 'cc_hist_tidy.csv'. You're all done!")
    print("You may now delete files {}".format(files_processed))
    print("they have been processed and added to the cc hist")
    
    return new

# Do it

In [16]:
filepath = 'C:/Users/geeze/Documents/finances/cc'

In [17]:
hist_files = glob.glob(filepath + '/*.csv')

In [18]:
#are these the right files?
hist_files

['C:/Users/geeze/Documents/finances/cc\\cc_hist_tidy.csv',
 'C:/Users/geeze/Documents/finances/cc\\chase8723_Activity20190101_20190919_20190919.CSV',
 'C:/Users/geeze/Documents/finances/cc\\citi-Year to date.CSV',
 'C:/Users/geeze/Documents/finances/cc\\discover-2019-YearToDateSummary.csv']

In [19]:
#full cc hist to update
full_cc_hist = open_full_hist([z for z in hist_files if 'tidy' in z][0])

In [28]:
#to cross reference against your cc hist sheet last entries
full_cc_hist.loc[(max(full_cc_hist.index)-10) : max(full_cc_hist.index)]

,date,price,desc,card,cat,st close,st open,Category
939,2019-07-19,5.00,Patreon* Membership INTERNET GBR,citi -6845,NaN,NaN,NaN,NaN
940,2019-07-20,85.82,FOUNDERS ALE HOUSE LOS ANGELES CA02048R,discover -1362,Restaurants,NaN,NaN,NaN
941,2019-07-20,217.18,FOAM E-Z INC WESTMINSTER CA,citi -6845,NaN,NaN,NaN,NaN
942,2019-07-21,25.00,THE STRONGHOLD CLIMBIN LOS ANGELES CA,citi -6845,NaN,NaN,NaN,NaN
943,2019-07-21,54.41,THE HOME DEPOT #6610 ALHAMBRA CA,citi -6845,NaN,NaN,NaN,NaN
944,2019-07-22,24.08,VONS #3075 STH PASADENA CA,citi -6845,NaN,NaN,NaN,NaN
945,2019-07-22,32.23,CHEVRON 0308292,chase -8723,NaN,NaN,NaN,Gas
946,2019-07-23,41.62,THE MENS WEARHOUSE #27 PASADENA CA,citi -6845,NaN,NaN,NaN,NaN
947,2019-07-25,22.22,VONS STORE 3075 SOUTH PASADENCA,discover -1362,Supermarkets,NaN,NaN,NaN
948,2019-07-27,76.64,THE MENS WEARHOUSE #27 PASADENA CA,citi -6845,NaN,NaN,NaN,NaN


In [29]:
new_stuff, files_processed, cleaned_new_cc_info = all_things_to_add(hist_files, full_cc_hist)

In [38]:
new_stuff.to_csv(filepath + '/new_stuff.csv')

In [39]:
for card_cleaned in cleaned_new_cc_info:
    card_cleaned.to_csv(filepath + '/{}.csv'.format(card_cleaned['card'].unique()))

In [40]:
save_new_cc_hist (full_cc_hist, new_stuff, filepath, files_processed)

The updated history of cc transactions has been saved to file 'cc_hist_tidy.csv'. You're all done!
You may now delete files ['C:/Users/geeze/Documents/finances/cc\\chase8723_Activity20190101_20190919_20190919.CSV', 'C:/Users/geeze/Documents/finances/cc\\citi-Year to date.CSV', 'C:/Users/geeze/Documents/finances/cc\\discover-2019-YearToDateSummary.csv']
they have been processed and added to the cc hist


,date,price,desc,card,cat,st close,st open,Category
0,2013-12-01,4.79,"//USC Income (2,000/mo). Open date unknown",amex -2002,NaN,12/27/2013,12/1/2013,NaN
1,2014-01-01,259.39,NaN,amex -2002,NaN,1/27/2014,12/28/2013,NaN
2,2014-02-01,343.67,NaN,amex -2002,NaN,2/24/2014,1/28/2014,NaN
3,2014-03-01,237.71,NaN,amex -2002,NaN,3/27/2014,2/25/2014,NaN
4,2014-04-01,288.67,NaN,amex -2002,NaN,4/25/2014,3/28/2014,NaN
5,2014-05-01,182.03,NaN,amex -2002,NaN,5/27/2014,4/26/2014,NaN
6,2014-06-01,475.33,NaN,amex -2002,NaN,6/26/2014,5/28/2014,NaN
7,2014-07-01,326.33,NaN,amex -2002,NaN,7/27/2014,6/27/2014,NaN
8,2014-08-01,346.95,NaN,amex -2002,NaN,8/27/2014,7/28/2014,NaN
9,2014-09-01,164.56,NaN,amex -2002,NaN,9/26/2014,8/28/2014,NaN
